# MC REINFORCE

In [1]:
'''
A bunch of imports, you don't have to worry about these
'''

import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import namedtuple, deque
import torch.optim as optim
import gym
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [2]:
'''
Please refer to the first tutorial for more details on the specifics of environments
We've only added important commands you might find useful for experiments.
'''

'''
List of example environments
(Source - https://gym.openai.com/envs/#classic_control)

'Acrobot-v1'
'Cartpole-v1'
'MountainCar-v0'
'''

env = gym.make('Acrobot-v1')
env.seed(0)

state_shape = env.observation_space.shape[0]
no_of_actions = env.action_space.n

print("State shape:", state_shape)
print("Number of Actions: ",no_of_actions)
print("Sampled Action",env.action_space.sample())
print("----")

'''
# Understanding State, Action, Reward Dynamics

The agent decides an action to take depending on the state.

The Environment keeps a variable specifically for the current state.
- Everytime an action is passed to the environment, it calculates the new state and updates the current state variable.
- It returns the new current state and reward for the agent to take the next action

'''

state = env.reset()
''' This returns the initial state (when environment is reset) '''

print("Current_State: ",state)
print("----")

action = env.action_space.sample()
''' We take a random action now '''

print("Sampled Action2: ", action)
print("----")

next_state, reward, done, info = env.step(action)
''' env.step is used to calculate new state and obtain reward based on old state and action taken  '''

print("Next_State: ",next_state)
print("Reward: ",reward)
print("Done: ", done)
print("Info: ", info)
print("----")


State shape: 6
Number of Actions:  3
Sampled Action 2
----
Current_State:  [ 0.99603073 -0.08901003  0.99567135  0.09294385  0.02653819 -0.04199653]
----
Sampled Action2:  0
----
Next_State:  [ 0.99829918 -0.05829878  0.99945086  0.03313578  0.27308215 -0.54190945]
Reward:  -1.0
Done:  False
Info:  {}
----


# MC REINFORCE W/O BASELINE

In [3]:

import torch
import torch.nn as nn
import torch.nn.functional as F

import random
import torch
import numpy as np
from collections import deque, namedtuple

from scipy.special import softmax
from torch.distributions import Categorical

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

'''
Bunch of Hyper parameters (Which you might have to tune later)
'''
'''BUFFER_SIZE = int(1e5)  # replay buffer size
BATCH_SIZE = 64         # minibatch size
GAMMA = 0.99            # discount factor
LR = 5e-4               # learning rate
UPDATE_EVERY = 20       # how often to update the network (When Q target is present)'''

class Policy(nn.Module):

    def __init__(self, state_size, action_size, seed=0, num_policy_layers =1,num_policy_layer_units= 64):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            seed (int): Random seed
            fc1_units (int): Number of nodes in first hidden layer
            fc2_units (int): Number of nodes in second hidden layer
        """
        super(Policy, self).__init__()
        self.seed = torch.manual_seed(seed)
        activation = nn.LeakyReLU
        self.fc_start = nn.Sequential(*[nn.Linear(state_size, num_policy_layer_units),activation()])
        self.fc_hidden = nn.Sequential(*[nn.Sequential(*[nn.Linear(num_policy_layer_units,num_policy_layer_units),activation()]) for _ in range(num_policy_layers-1)])
        self.fc_end = nn.Linear(num_policy_layer_units, action_size)

    def forward(self, state):
        """Build a network that maps state -> action values."""
        xs = self.fc_start(state)
        x_hidden = self.fc_hidden(xs)
        x_out = self.fc_end(x_hidden)
        return F.softmax(x_out, dim=1)
    
    def act(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = self.forward(state).cpu()
        model = Categorical(probs)
        action = model.sample()
        return action.item(), model.log_prob(action)


In [4]:
class Baseline(nn.Module):
    def __init__(self, state_size, seed=0, num_baseline_layers =1,num_baseline_layer_units= 64):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            seed (int): Random seed
            fc1_units (int): Number of nodes in first hidden layer
            fc2_units (int): Number of nodes in second hidden layer
        """
        super(Baseline, self).__init__()
        self.seed = torch.manual_seed(seed)
        activation = nn.LeakyReLU
        self.fc_start = nn.Sequential(*[nn.Linear(state_size, num_baseline_layer_units),activation()])
        self.fc_hidden = nn.Sequential(*[nn.Sequential(*[nn.Linear(num_baseline_layer_units,num_baseline_layer_units),activation()]) for _ in range(num_baseline_layers-1)])
        self.fc_end = nn.Linear(num_baseline_layer_units, 1)

    def forward(self, state):
        """Build a network that maps state -> action values."""
        xs = self.fc_start(state)
        x_hidden = self.fc_hidden(xs)
        x_out = self.fc_end(x_hidden)
        return x_out

In [5]:
def reinforce_with_baseline(policy, baseline, optimizer_policy, optimizer_baseline, n_episodes=1000, max_t=1000, gamma=0.99, print_every=100):
    scores_deque = deque(maxlen=100)
    scores = []
    for e in range(n_episodes):
        saved_log_probs = []
        rewards = []
        states = []
        state = env.reset()
        # Collect trajectory
        for t in range(max_t):
            action, log_prob = policy.act(state)
            saved_log_probs.append(log_prob)
            state, reward, done, _ = env.step(action)
            rewards.append(reward)
            states.append(state)
            if done:
                break
        scores_deque.append(sum(rewards))
        scores.append(sum(rewards))

        # Convert trajectory to torch tensors
        states = torch.FloatTensor(np.array(states)).to(device)
        rewards = torch.FloatTensor(rewards)

        # Calculate discounted returns
        discounts = torch.FloatTensor([gamma ** i for i in range(len(rewards))])
        returns = torch.FloatTensor([sum(rewards[i:] * discounts[:len(rewards)-i]) for i in range(len(rewards))]).to(device)

        # Calculate baseline values and advantages
        baseline_values = baseline(states).squeeze()
        advantages = returns - baseline_values

        # Calculate policy loss
        policy_loss = [-log_prob * advantage.detach().cpu() for log_prob, advantage in zip(saved_log_probs, advantages)]
        policy_loss = torch.cat(policy_loss).sum()

        # Calculate baseline loss
        baseline_loss = F.mse_loss(baseline_values, returns)

        # Backpropagation and optimization step
        optimizer_policy.zero_grad()
        policy_loss.backward()
        optimizer_policy.step()

        optimizer_baseline.zero_grad()
        baseline_loss.backward()
        optimizer_baseline.step()

        if e % print_every == 0:
            print('Episode {}\tAverage Score: {:.2f}'.format(e, np.mean(scores_deque)))
    return scores

# Final Draft

In [6]:
#!pip install wandb

In [7]:
import wandb
wandb.login(key = "8545e71f98dc96fbac53295facb12404fc77016d")

wandb: Currently logged in as: nayinisriharsh-iitm. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\nayin\.netrc


True

In [8]:
def train_and_tune(config=None):
  # Initialize a new wandb run
  with wandb.init(config=config):
    # If called by wandb.agent, as below,
    # this config will be set by Sweep Controller
    config = wandb.config
    wandb.run.name='plr'+str(config.policy_lr)+'blr'+str(config.baseline_lr)+'-nlr'+str(config.num_policy_layers)+'-uts'+str(config.num_policy_layer_units)+'-nlr'+str(config.num_baseline_layers)+'-uts'+str(config.num_baseline_layer_units)
    state_shape = env.observation_space.shape[0]
    action_shape = env.action_space.n
    num_exp = 1
    max_episodes = 1000
    total_rewards = np.zeros([num_exp,max_episodes])
    for i in range(num_exp):
        state_shape = env.observation_space.shape[0]
        action_shape = env.action_space.n
        
        policy = Policy(state_size=state_shape, action_size=action_shape, seed=i, num_policy_layers =config.num_policy_layers,num_policy_layer_units= config.num_policy_layer_units).to(device)
        baseline = Baseline(state_size=state_shape, seed=i, num_baseline_layers =config.num_baseline_layers,num_baseline_layer_units= config.num_baseline_layer_units).to(device)
        optimizer_policy = optim.Adam(policy.parameters(), lr=config.policy_lr)
        optimizer_baseline = optim.Adam(baseline.parameters(), lr=config.baseline_lr)
        scores = reinforce_with_baseline(policy, baseline, optimizer_policy, optimizer_baseline, n_episodes=max_episodes)
        total_rewards[i] = scores
    Regret = np.mean(np.sum(0-total_rewards,axis=1),axis=0)
    #wandb.log({"train_mean_reward":rewards,"train_mean_steps":steps,"test_mean_reward":r,"train_mean_steps":s})
    #data = [[x, y] for (x, y) in zip(np.arange(config.episodes), rewards)]
    #table1 = wandb.Table(data=data, columns=["x", "y"])
    #data = [[x, y] for (x, y) in zip(np.arange(config.episodes), steps)]
    #table2 = wandb.Table(data=data, columns=["x", "y"])
    wandb.log(
        {
            "avg_regret":Regret#,'avg_test_steps':s #,"train_reward": wandb.plot.line(table1, "x", "y", title="Reward vs Episode"),"train_steps": wandb.plot.line(table2, "x", "y", title="Steps vs Episode"),
        }
    )

In [9]:
sweep_config={'method':'bayes',
              'metric' : {
                  'name':'avg_regret',
                  'goal':'minimize'},
              'parameters':{
                  'policy_lr':{'values':[1e-2, 1e-3, 1e-4]},
                  'baseline_lr':{'values':[1e-2, 1e-3, 1e-4]},
                  'num_policy_layers':{'values':[1,2,3]},
                  'num_policy_layer_units':{'values':[64,128,256]},
                  'num_baseline_layers':{'values':[1,2,3]},
                  'num_baseline_layer_units':{'values':[64,128,256]},
                  }}
import pprint
pprint.pprint(sweep_config)
sweep_id=wandb.sweep(sweep_config,project="CS6700_PROGRAMMING_ASSIGNMENT_2")

{'method': 'bayes',
 'metric': {'goal': 'minimize', 'name': 'avg_regret'},
 'parameters': {'baseline_lr': {'values': [0.01, 0.001, 0.0001]},
                'num_baseline_layer_units': {'values': [64, 128, 256]},
                'num_baseline_layers': {'values': [1, 2, 3]},
                'num_policy_layer_units': {'values': [64, 128, 256]},
                'num_policy_layers': {'values': [1, 2, 3]},
                'policy_lr': {'values': [0.01, 0.001, 0.0001]}}}
Create sweep with ID: 2puxr4da
Sweep URL: https://wandb.ai/nayinisriharsh-iitm/CS6700_PROGRAMMING_ASSIGNMENT_2/sweeps/2puxr4da


{'method': 'bayes',
 'metric': {'goal': 'minimize', 'name': 'avg_regret'},
 'parameters': {'baseline_lr': {'values': [0.01, 0.001, 0.0001]},
                'num_baseline_layer_units': {'values': [64, 128, 256]},
                'num_baseline_layers': {'values': [1, 2, 3]},
                'num_policy_layer_units': {'values': [64, 128, 256]},
                'num_policy_layers': {'values': [1, 2, 3]},
                'policy_lr': {'values': [0.01, 0.001, 0.0001]}}}
Create sweep with ID: shtmetho
Sweep URL: https://wandb.ai/nayinisriharsh-iitm/CS6700_PROGRAMMING_ASSIGNMENT_2/sweeps/i63qbi5w

In [ ]:
wandb.agent(sweep_id, train_and_tune,count=40)

wandb: Agent Starting Run: 8w7hvf6x with config:
wandb: 	baseline_lr: 0.001
wandb: 	num_baseline_layer_units: 64
wandb: 	num_baseline_layers: 2
wandb: 	num_policy_layer_units: 256
wandb: 	num_policy_layers: 2
wandb: 	policy_lr: 0.01


Episode 0	Average Score: -500.00
Episode 100	Average Score: -460.99
Episode 200	Average Score: -500.00
Episode 300	Average Score: -500.00
Episode 400	Average Score: -500.00
Episode 500	Average Score: -500.00
Episode 600	Average Score: -500.00
Episode 700	Average Score: -500.00
Episode 800	Average Score: -500.00
Episode 900	Average Score: -500.00


avg_regret,▁
avg_regret,496099.0


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: qf3esqfw with config:
wandb: 	baseline_lr: 0.001
wandb: 	num_baseline_layer_units: 256
wandb: 	num_baseline_layers: 1
wandb: 	num_policy_layer_units: 256
wandb: 	num_policy_layers: 1
wandb: 	policy_lr: 0.001


Episode 0	Average Score: -500.00
Episode 100	Average Score: -488.84
Episode 200	Average Score: -165.53
Episode 300	Average Score: -120.40
Episode 400	Average Score: -108.88
Episode 500	Average Score: -107.67
Episode 600	Average Score: -106.43
Episode 700	Average Score: -112.06
Episode 800	Average Score: -116.06
Episode 900	Average Score: -114.18


...

wandb: Agent Starting Run: jmw3ahwc with config:
wandb: 	baseline_lr: 0.001
wandb: 	num_baseline_layer_units: 256
wandb: 	num_baseline_layers: 1
wandb: 	num_policy_layer_units: 256
wandb: 	num_policy_layers: 2
wandb: 	policy_lr: 0.0001


Episode 0	Average Score: -500.00
Episode 100	Average Score: -492.79
Episode 200	Average Score: -279.18
Episode 300	Average Score: -160.43
Episode 400	Average Score: -135.04
Episode 500	Average Score: -133.11
Episode 600	Average Score: -125.02
Episode 700	Average Score: -131.65
Episode 800	Average Score: -126.33
Episode 900	Average Score: -123.72


avg_regret,▁
avg_regret,183414.0


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: ez1aiqyx with config:
wandb: 	baseline_lr: 0.01
wandb: 	num_baseline_layer_units: 256
wandb: 	num_baseline_layers: 1
wandb: 	num_policy_layer_units: 256
wandb: 	num_policy_layers: 2
wandb: 	policy_lr: 0.0001


Episode 0	Average Score: -500.00
Episode 100	Average Score: -360.35
Episode 200	Average Score: -217.21
Episode 300	Average Score: -218.87
Episode 400	Average Score: -206.79
Episode 500	Average Score: -230.02
Episode 600	Average Score: -195.32
Episode 700	Average Score: -209.24
Episode 800	Average Score: -229.48
Episode 900	Average Score: -238.07
